# Laboratorio 3
Ignacio Mendez (22613) y Diego Soto (22737)

## Task 1

#### Carga de librerias

In [7]:
import numpy as np
import math
import cv2
import time

#### Matrices a usar

In [2]:
M = np.array([
    [120, 5],
    [5, 115]
])

M2 = np.array([
    [200, 10],
    [10, 1]
])


#### Creación de funciones

In [ ]:
def autovalores_2x2(A):

    a = A[0][0]
    b = A[0][1]
    c = A[1][0]
    d = A[1][1]

    tr = a + d
    det = a*d - b*c

    disc = tr**2 - 4*det

    lambda1 = (tr + math.sqrt(disc)) / 2
    lambda2 = (tr - math.sqrt(disc)) / 2

    return lambda1, lambda2


In [4]:
def harris(M, k=0.04):
    m11 = M[0][0]
    m12 = M[0][1]
    m22 = M[1][1]

    detM = m11 * m22 - m12 * m12
    trM = m11 + m22

    R = detM - k * (trM ** 2)
    return R


#### Matriz M

$$
M =
\begin{pmatrix}
120 & 5 \\
5 & 115
\end{pmatrix}
$$

### Eigenvalores

$$
\operatorname{tr}(M) = 120 + 115 = 235
$$

$$
\det(M) = 120 \cdot 115 - 5 \cdot 5 = 13775
$$

$$
\operatorname{tr}(M)^2 - 4\det(M) = 235^2 - 4(13775) = 125
$$

$$
\sqrt{125} \approx 11.1803
$$

$$
\lambda_1 = \frac{235 + 11.1803}{2} \approx 123.0902
$$

$$
\lambda_2 = \frac{235 - 11.1803}{2} \approx 111.9098
$$

### Respuesta de Harris

$$
R = \det(M) - k(\operatorname{tr}(M))^2
$$

$$
R = 13775 - 0.04(235^2)
$$

$$
R = 13775 - 2209 = 11566
$$

In [6]:
eig_M = autovalores_2x2(M)
harris_M = harris(M)

print("Autovalores matriz M:", eig_M)
print("Respuesta de Harris matriz M:", harris_M)

Autovalores matriz M: (123.09016994374947, 111.90983005625053)
Respuesta de Harris matriz M: 11566.0


Para la matriz M se puede ver que ambos autovalores son altos indicando cambios fuertes en ambas direcciones, la respuesta de Harris es positiva y alta. Esto indica que el pixel analizado en la matriz M es una **esquina**. 

#### Matriz M'

$$
M' =
\begin{pmatrix}
200 & 10 \\
10 & 1
\end{pmatrix}
$$

### Eigenvalores

$$
\operatorname{tr}(M') = 200 + 1 = 201
$$

$$
\det(M') = 200 \cdot 1 - 10 \cdot 10 = 100
$$

$$
\operatorname{tr}(M')^2 - 4\det(M') = 201^2 - 4(100) = 40001
$$

$$
\sqrt{40001} \approx 200.0025
$$

$$
\lambda_1 = \frac{201 + 200.0025}{2} \approx 200.50125
$$

$$
\lambda_2 = \frac{201 - 200.0025}{2} \approx 0.49875
$$

### Respuesta de Harris

$$
R = \det(M') - k(\operatorname{tr}(M'))^2
$$

$$
R = 100 - 0.04(201^2)
$$

$$
R = 100 - 1616.04 = -1516.04
$$

In [7]:
eig_M2 = autovalores_2x2(M2)
harris_M2 = harris(M2)

print("Autovalores matriz M': ", eig_M2)
print("Respuesta de Harris matriz M': ", harris_M2)

Autovalores matriz M':  (200.50124999218758, 0.4987500078124043)
Respuesta de Harris matriz M':  -1516.04


Para el caso de la matriz M' se puede ver que un autovalor es grande y el otro pequeño, indicando cambio fuerte en una sola dirección, la respuesta de Harris es negativa. Esto indica que el pixel analizado en la matriz M' es un **borde**. 

## Task 2

In [2]:
imagen1 = cv2.imread("imagen1.png", cv2.IMREAD_GRAYSCALE)
imagen2 = cv2.imread("imagen2.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
# Creamos el detector SIFT
sift = cv2.SIFT_create()

# Se detectan keypoints y descriptores en las dos imágenes, o sea la frontal y la girada
kp1_sift, des1_sift = sift.detectAndCompute(imagen1, None)
kp2_sift, des2_sift = sift.detectAndCompute(imagen2, None)

# BFMatcher con distancia Euclidiana
bf_sift = cv2.BFMatcher(cv2.NORM_L2)

# Se buscan los dos mejores matches por descriptor
matches_sift = bf_sift.knnMatch(des1_sift, des2_sift, k=2)

buen_sift = []

for m, n in matches_sift:

    if m.distance < 0.75 * n.distance:
        buen_sift.append(m)

imagen_sift = cv2.drawMatches(
    imagen1, kp1_sift,
    imagen2, kp2_sift,
    buen_sift, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

In [ ]:
# Creamos el detector ORB
orb = cv2.ORB_create(nfeatures=2000)

kp1_orb, des1_orb = orb.detectAndCompute(imagen1, None)
kp2_orb, des2_orb = orb.detectAndCompute(imagen2, None)

# BFMatcher con distancia Hamming
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING)

matches_orb = bf_orb.knnMatch(des1_orb, des2_orb, k=2)

buen_orb = []

for m, n in matches_orb:
    if m.distance < 0.75 * n.distance:
        buen_orb.append(m)

imagen_orb = cv2.drawMatches(
    imagen1, kp1_orb,
    imagen2, kp2_orb,
    buen_orb, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

In [ ]:
# Le cambiamos el tamaño para que se vea mejor la imagen
def mostrar_imagen(img, nombre, ancho=1200):
    h, w = img.shape[:2]
    escala = ancho / w
    nueva_dim = (ancho, int(h * escala))
    img_resized = cv2.resize(img, nueva_dim)
    cv2.imshow(nombre, img_resized)


In [6]:
mostrar_imagen(imagen_sift, "Matches con SIFT")
mostrar_imagen(imagen_orb, "Matches con ORB")

cv2.waitKey(0)
cv2.destroyAllWindows()

## Task 3

In [ ]:
# SIFT
# Usamos time para medir los tiempos de ejecución
t0 = time.time()

# Creamos el detector SIFT
sift = cv2.SIFT_create()

# Se detectan keypoints y descriptores en las dos imágenes, o sea la frontal y la girada
kp1_sift, des1_sift = sift.detectAndCompute(imagen1, None)
kp2_sift, des2_sift = sift.detectAndCompute(imagen2, None)

t1 = time.time()

# BFMatcher con distancia Euclidiana
bf_sift = cv2.BFMatcher(cv2.NORM_L2)

# Se buscan los dos mejores matches por descriptor
matches_sift = bf_sift.knnMatch(des1_sift, des2_sift, k=2)

buen_sift = []
for m, n in matches_sift:
    if m.distance < 0.75 * n.distance:
        buen_sift.append(m)

t2 = time.time()

tiempo_sift_detect = (t1 - t0) * 1000
tiempo_sift_match = (t2 - t1) * 1000
tiempo_sift_total = (t2 - t0) * 1000

In [ ]:
# ORB
t0 = time.time()

# Creamos el detector ORB
orb = cv2.ORB_create(nfeatures=2000)

kp1_orb, des1_orb = orb.detectAndCompute(imagen1, None)
kp2_orb, des2_orb = orb.detectAndCompute(imagen2, None)

t1 = time.time()

# BFMatcher con distancia Hamming
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING)

matches_orb = bf_orb.knnMatch(des1_orb, des2_orb, k=2)

buen_orb = []
for m, n in matches_orb:
    if m.distance < 0.75 * n.distance:
        buen_orb.append(m)

t2 = time.time()

tiempo_orb_detect = (t1 - t0) * 1000
tiempo_orb_match = (t2 - t1) * 1000
tiempo_orb_total = (t2 - t0) * 1000

In [ ]:
print("SIFT")
print("Keypoints detectados en la imagen1:", len(kp1_sift))
print("Keypoints detectados en la imagen2:", len(kp2_sift))
print("Matches 'buenos':", len(buen_sift))
print("Tiempo total (ms):", tiempo_sift_total)

print("\nORB")
print("Keypoints detectados en la imagen1:", len(kp1_orb))
print("Keypoints detectados en la imagen2:", len(kp2_orb))
print("Matches 'buenos':", len(buen_orb))
print("Tiempo total (ms):", tiempo_orb_total)


SIFT
Keypoints detectados en la imagen1: 5755
Keypoints detectados en la imagen2: 4987
Matches 'buenos': 1152
Tiempo total (ms): 966.7224884033203

ORB
Keypoints detectados en la imagen1: 2000
Keypoints detectados en la imagen2: 2000
Matches 'buenos': 338
Tiempo total (ms): 69.55289840698242


**a. ¿Cuál algoritmo elegiría para el Producto A (Drone de Carreras) y por qué? Base su respuesta en los milisegundos que mediste y la tasa de refresco requerida (60 FPS ≈ 16ms de presupuesto total).**

Elegiría ORB porque, aunque tiene menos matches que SIFT, es significativamente más rápido. SIFT tardó ~966.72 ms y ORB ~69.55 ms, y para un sistema que requiere 60 FPS (~16 ms por frame), ORB es el único viable de optimizar. En un drone es importante la velocidad sobre la precisión extrema, y ORB permite acercarse más a tiempo real reduciendo resolución o keypoints.


**b. ¿Cuál algoritmo elegirías para el Producto B (Inspección) y por qué? Analice la calidad visual de los matches en los cambios de escala y rotación que probaste. ¿Falló ORB en algún caso donde SIFT tuvo éxito?**

Elegiría SIFT porque ofrece mejor calidad visual en los matches, especialmente frente a cambios de escala y rotación. En las pruebas, SIFT produjo muchos más matches correctos (1152 vs 338), mientras que ORB perdió correspondencias en casos complejos donde SIFT sí logró emparejar bien. Para inspección (producto B), donde la precisión es indispensable y el tiempo no es tan limitado, SIFT es la mejor opción.


**c. ¿Las conclusiones que estamos alcanzando son justas y generalizables? ¿Por qué? ¿Qué deberíamos considerar en futuras iteraciones?**

Las conclusiones son razonables pero no completamente generalizables, porque se basan en un conjunto limitado de imágenes y parámetros. Los resultados dependen de la resolución, el tipo de escenario, la iluminación y la implementación. En futuras iteraciones se debería probar con más escenarios, ajustar parámetros y evaluar métricas adicionales como precisión de matches.